# MUC5B Mutation Trans Effect on Proteomics

This notebook analyzes the trans effect of MUC5B mutation on interacting and other proteins, in Endometrial and Colon cancer.

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import collections
import re
import gseapy as gp
from gseapy.plot import barplot, dotplot
import matplotlib.pyplot as plt
import seaborn as sns

import cptac
import cptac.algorithms as al

en = cptac.Endometrial()
co = cptac.Colon()
ov = cptac.Ovarian()

### Specify Gene

In [2]:
gene = "MUC5B"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
omics = "proteomics"
#omics = "transcriptomics"
#omics = "phosphoproteomics"
#omics = "acetylproteomics"

# Interacting Proteins: proteomics

## Endometrial

### Generate interacting protein list

In [4]:
'''Use get interacting proteins method to generate list of interacting proteins'''

#comment about how this returns ranked interacting proteins (F/ uniprot or string)
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = en.get_proteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
SMARCC1
SUPT16H
CCND1
SMARCD3
CREBBP
SMARCB1
KAT2B
DNMT3A
SMARCA2
SMARCC2
KMT2D
ARID1A
SMARCE1
PHF10
BAZ1B
ARID1B
NF1
CDC5L
SMARCD1
SMARCD2
SMARCA4
ACTL6A
BCL7B
SS18
DPF2
DPF3
ING1
PGR
BCL7C
HIC1


### Test for significant comparisons in any of interacting proteins

In [5]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = en.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

Doing t-test comparisons

           Comparison       P_Value
0   ARID1A_proteomics  1.025018e-10
1     DPF2_proteomics  2.166609e-07
2  SMARCB1_proteomics  6.885135e-05
3  SMARCC2_proteomics  8.270473e-05
4  SMARCE1_proteomics  1.498027e-04
5  SMARCD1_proteomics  1.744915e-04
6    BCL7C_proteomics  4.379537e-04
7    CCND1_proteomics  1.499832e-03





## Colon

### Generate interacting protein list

In [6]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = co.get_proteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
SMARCC1
SUPT16H
CREBBP
SMARCB1
SMARCA2
SMARCC2
ARID1A
SMARCE1
PHF10
BAZ1B
ARID1B
NF1
CDC5L
SMARCD1
SMARCD2
SMARCA4
ACTL6A
BCL7B
DPF2
ING1
BCL7C


### Test for significant comparisons in any of interacting proteins

In [7]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = co.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)

'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")

Doing t-test comparisons

          Comparison   P_Value
0   BAZ1B_proteomics  0.000841
1  ARID1A_proteomics  0.001305





## Ovarian

### Generate interacting protein list

In [8]:
'''Use get interacting proteins method to generate list of interacting proteins'''
interacting_proteins = al.get_interacting_proteins(gene)

omics_object = ov.get_proteomics()

print("Generating interacting protein list")
interacting_proteins_in_omics_df = []

'''Only do comparisons on proteins in the omics dataframe'''
for ip in interacting_proteins:
    if omics == 'phosphoproteomics' or omics == 'acetylproteomics':
        col_regex = ip + "-.*" # Build a regex to get all columns that match the gene
    else:
        col_regex = '^{}$'.format(ip)

    selected = omics_object.filter(regex=col_regex)

    if len(selected.columns) > 0:
        interacting_proteins_in_omics_df.append(ip)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins_in_omics_df:
    print(interacting_protein)

Generating interacting protein list
Interacting Proteins:
SMARCC1
SUPT16H
SMARCD3
CREBBP
SMARCB1
KAT2B
DNMT3A
SMARCA2
SMARCC2
KMT2D
ARID1A
SMARCE1
PHF10
BAZ1B
ARID1B
NF1
CDC5L
SMARCD1
SMARCD2
SMARCA4
ACTL6A
BCL7B
SS18
DPF2
KLF1
ING1
PGR
BCL7C
HIC1


### Test for significant comparisons in any of interacting proteins

In [9]:
'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = ov.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins_in_omics_df)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']


'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.loc[ind,'Label'] = 'Mutated'
    else:
        protdf.loc[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)
#protdf = protdf.drop_duplicates()
protdf = protdf.loc[:,~protdf.columns.duplicated()]


'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")


'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")


Doing t-test comparisons



/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


        Comparison       P_Value
0  ING1_proteomics  9.149984e-07





# All Proteins: proteomics

## Endometrial

In [10]:
try:
    print("\nGene: ", gene)

    '''Use all proteins'''
    proteomics = en.get_proteomics()
    all_proteins = proteomics.columns
    #all_proteins = all_proteins[:100]

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = en.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the datafram correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  ARID1A
Doing t-test comparisons

          Comparison       P_Value
0  ARID1A_proteomics  1.025018e-10
1   MAP1S_proteomics  1.010149e-07
2    DPF2_proteomics  2.166609e-07
3   KDM4B_proteomics  4.414547e-07
4    RRM1_proteomics  4.455870e-07
5  MAP7D3_proteomics  4.880890e-07
6   CYTH3_proteomics  3.109396e-06
7   CEP78_proteomics  3.749172e-06





## Colon

In [11]:
try:
    print("\nGene: ", gene)
    
    proteomics = co.get_proteomics()
    '''Use all proteins'''
    all_proteins = proteomics.columns
    #all_proteins = all_proteins[:100]

    '''Create dataframe in order to do comparisons with wrap_ttest'''
    protdf = co.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    '''Create the binary valued column needed to do the comparison'''
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    '''Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    '''Make list of columns to be compared using t-tests'''
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    '''Call wrap_ttest, pass in formatted dataframe'''
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    '''Print results, if anything significant was found'''
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")

except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  ARID1A
Doing t-test comparisons

No significant comparisons.


## Ovarian

In [12]:

print("\nGene: ", gene)

'''Get proteomics df'''
proteomics = ov.get_proteomics()

'''Use all proteins'''
all_proteins = proteomics.columns
all_proteins = list(set(all_proteins))

'''Create dataframe in order to do comparisons with wrap_ttest'''
protdf = ov.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=all_proteins)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']


'''Create the binary valued column needed to do the comparison'''
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.loc[ind,'Label'] = 'Mutated'
    else:
        protdf.loc[ind,'Label'] = 'Wildtype'

'''Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)
protdf = protdf.loc[:,~protdf.columns.duplicated()]


'''Make list of columns to be compared using t-tests'''
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

'''Call wrap_ttest, pass in formatted dataframe'''
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

'''Print results, if anything significant was found'''
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")


Gene:  ARID1A
Doing t-test comparisons

           Comparison       P_Value
0    OLFM4_proteomics  4.658204e-07
1    TIMP1_proteomics  5.699514e-07
2   PAXIP1_proteomics  5.812213e-07
3     ING1_proteomics  9.149984e-07
4   ZBTB40_proteomics  2.658964e-06
5     PIGR_proteomics  2.712638e-06
6  TMEM63A_proteomics  3.925646e-06



